In [ ]:
from nilearn import datasets
import pandas as pd
import numpy as np
from nilearn import datasets
from nilearn import plotting
import os
import glob
from joblib import Parallel, delayed
from nilearn.connectome import ConnectivityMeasure
from nilearn.interfaces.fmriprep import load_confounds
from nilearn.maskers import NiftiLabelsMasker, MultiNiftiLabelsMasker
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
## Set fMRIprep Directory

In [ ]:
fmriprep_dir = ''
nil_data_dir = os.path.join(os.getcwd(), 'nilearn_data')

## Load Atlas

In [ ]:
from nilearn import datasets
atlas = datasets.fetch_atlas_schaefer_2018(n_rois=400, 
                                           yeo_networks=7, 
                                           resolution_mm=1, 
                                           data_dir=None,
                                           base_url=None, 
                                           resume=True,
                                           data_dir=nil_data_dir
                                           verbose=1)

In [ ]:
import nibabel as nib
nii = nib.load('{}/schaefer_2018/Schaefer2018_400Parcels_7Networks_order_FSLMNI152_1mm.nii.gz'.format(nil_data_dir))
nii_arr = nii.get_fdata()

In [ ]:
# Loading atlas image stored in 'maps'
atlas_filename = atlas['maps']
# Loading atlas data stored in 'labels'
labels = atlas['labels']

In [ ]:
plotting.plot_roi(atlas_filename, title="Schaefer Atlas")
plotting.show()

## Connectivity Matrices

In [ ]:
%%time
bolds = glob.glob('{}/*/ses-0*/func/*_ses-*_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'.format(fmriprep_dir)
bolds = [i for i in bolds if not 'NeuroMET074' in i] # Exclude one subject
sids = [i.split('/')[-4] for i in bolds]
ses = [i.split('/')[-3] for i in bolds]
correlation_measure = ConnectivityMeasure(kind='correlation')
masker = MultiNiftiLabelsMasker(atlas_filename, 
                         memory=os.path.join(nil_data_dir, 'schaefer_atlas'), 
                         standardize='zscore_sample',
                        smoothing_fwhm=6,
                         low_pass=0.1,
                         high_pass=0.008,
                         t_r=2.2,
                        memory_level=5)
confounds_simples, sample_masks = load_confounds(
            bolds, 
            strategy=['motion', 'compcor', 'scrub', 'global_signal', 'high_pass'],
            motion = 'full', global_signal='full', 
            scrub=0, compcor='anat_separated', n_compcor=5,
            ica_aroma='basic')
bolds_tmp = bolds
time_series_list = masker.fit_transform(bolds_tmp, confounds=confounds_simples)
cor_mat_tmp = correlation_measure.fit_transform(time_series_list)
#correlation_matrices.append(cor_mat_tmp)
np.savez('./matrices_ses-all_schaefer2018.npz', mats = cor_mat_tmp, sids = sids, ses=ses)
del(time_series_list)
for n,i in enumerate(cor_mat_tmp):
    print(sids[n])
    plotting.plot_matrix(i.squeeze())
    plotting.show()

## Networks' average functional Connectivity

### grab labels' indices for each network

In [ ]:
import re
labelsd = np.char.decode(labels)
nets = set([re.search(r'7Networks_[LR]H_([^_]*)_.*',i.decode()).group(1) for i in labels])
nets

In [ ]:
net_dict = dict()
for i in nets:
    net_dict[i] = []

In [ ]:
for n,i in enumerate(labelsd):
    for net in nets:
        if net in i:
            net_dict[net].append(n)

In [ ]:
len(cor_mat_tmp)

### Average functional connectivity averages

In [ ]:
conns = []
for n,mat in enumerate(cor_mat_tmp):
    conn_dict = dict()
    conn_dict['sids'] = sids[n]
    conn_dict['ses'] = ses[n]
    for i in nets:
        #print(net_dict[i])
        submat = mat[net_dict[i],:][:,net_dict[i]]
        submat_tril = np.tril(submat, k=-1)
        #submat_tril_thr = np.where(submat_tril <0, 0, submat_tril)
        submat_tril[np.triu_indices(submat_tril.shape[0],0)] = np.nan
        submat_tril_thr = np.where(submat_tril <0, 0, submat_tril)
        conn_dict[i] = np.nanmean(submat_tril_thr[submat_tril_thr>0])
    
    conns.append(conn_dict)
    

In [ ]:
pd.DataFrame(conns)

In [ ]:
pd.DataFrame(conns).to_csv('conns.csv', sep=';', index=False)